In [ ]:
!unzip /content/drive/MyDrive/covid_dataset.zip -d /content/drive/MyDrive

Archive:  /content/drive/MyDrive/covid_dataset.zip
  inflating: /content/drive/MyDrive/Covid19-dataset/test/Covid/0100.jpeg  
  inflating: /content/drive/MyDrive/Covid19-dataset/test/Covid/0102.jpeg  
  inflating: /content/drive/MyDrive/Covid19-dataset/test/Covid/0105.png  
  inflating: /content/drive/MyDrive/Covid19-dataset/test/Covid/0106.jpeg  
  inflating: /content/drive/MyDrive/Covid19-dataset/test/Covid/0108.jpeg  
  inflating: /content/drive/MyDrive/Covid19-dataset/test/Covid/0111.jpg  
  inflating: /content/drive/MyDrive/Covid19-dataset/test/Covid/0112.jpg  
  inflating: /content/drive/MyDrive/Covid19-dataset/test/Covid/0113.jpg  
  inflating: /content/drive/MyDrive/Covid19-dataset/test/Covid/0115.jpeg  
  inflating: /content/drive/MyDrive/Covid19-dataset/test/Covid/0118.jpeg  
  inflating: /content/drive/MyDrive/Covid19-dataset/test/Covid/0119.jpeg  
  inflating: /content/drive/MyDrive/Covid19-dataset/test/Covid/0120.jpg  
  inflating: /content/drive/MyDrive/Covid19-dataset/te

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = [224, 224] # the image of art take size of [224,224] thats why we take here.

train_path = '/content/drive/MyDrive/Covid19-dataset/train'
valid_path = '/content/drive/MyDrive/Covid19-dataset/test'

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
for layer in vgg.layers:
  layer.trainable=False

In [ ]:
folders=glob( '/content/drive/MyDrive/Covid19-dataset/train/*')

In [ ]:
x=Flatten()(vgg.output)

prediction=Dense(len(folders),activation='softmax')(x)


model = Model(inputs=vgg.input, outputs=prediction)


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Covid19-dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


Found 251 images belonging to 3 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Covid19-dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 66 images belonging to 3 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
8/8 [==============================] - 188s 23s/step - loss: 1.3109 - accuracy: 0.5000 - val_loss: 0.4077 - val_accuracy: 0.7879
Epoch 2/10
8/8 [==============================] - 179s 23s/step - loss: 0.3598 - accuracy: 0.8480 - val_loss: 0.2259 - val_accuracy: 0.9091
Epoch 3/10
8/8 [==============================] - 172s 22s/step - loss: 0.2122 - accuracy: 0.9021 - val_loss: 0.1674 - val_accuracy: 0.9545
Epoch 4/10
8/8 [==============================] - 173s 22s/step - loss: 0.1757 - accuracy: 0.9564 - val_loss: 0.1330 - val_accuracy: 0.9697
Epoch 5/10
8/8 [==============================] - 176s 22s/step - loss: 0.0898 - accuracy: 0.9795 - val_loss: 0.1142 - val_accuracy: 0.9545
Epoch 6/10
8/8 [==============================] - 173s 22s/step - loss: 0.1025 - accuracy: 0.9704 - val_loss: 0.1032 - val_accuracy: 1.0000
Epoch 7/10
8/8 [==============================] - 174s 22s/step - loss: 0.0563 - accuracy: 0.9980 - val_loss: 0.1135 - val_accuracy: 0.9545
Epoch 8/10
8/8 [====

In [ ]:
r.history

{'accuracy': [0.6454183459281921,
  0.8884462118148804,
  0.8964143395423889,
  0.9601593613624573,
  0.9760956168174744,
  0.980079710483551,
  0.9960159659385681,
  0.9960159659385681,
  0.9840637445449829,
  0.9920318722724915],
 'loss': [1.0169700384140015,
  0.297542929649353,
  0.21434564888477325,
  0.15117348730564117,
  0.09586728364229202,
  0.0859241634607315,
  0.0594683401286602,
  0.050998810678720474,
  0.057373303920030594,
  0.05512377992272377],
 'val_accuracy': [0.7878788113594055,
  0.9090909361839294,
  0.9545454382896423,
  0.9696969985961914,
  0.9545454382896423,
  1.0,
  0.9545454382896423,
  1.0,
  0.9545454382896423,
  0.9848484992980957],
 'val_loss': [0.407712459564209,
  0.22590406239032745,
  0.1673826277256012,
  0.1329890936613083,
  0.11416540294885635,
  0.10317893326282501,
  0.11348167061805725,
  0.09326566010713577,
  0.0933995395898819,
  0.09206879138946533]}

In [ ]:
from tensorflow.keras.models import load_model

model.save('covid_vgg.h5')

In [ ]:
y_pred=model.predict(test_set)
y_pred

array([[9.97004807e-01, 3.04532557e-04, 2.69063865e-03],
       [2.06605229e-03, 2.93256968e-01, 7.04676986e-01],
       [9.98009145e-01, 6.43818581e-04, 1.34702702e-03],
       [9.98578429e-01, 1.84208784e-05, 1.40317052e-03],
       [2.22084273e-04, 9.93510425e-01, 6.26746425e-03],
       [1.71336185e-04, 3.53205204e-02, 9.64508116e-01],
       [9.98497367e-01, 2.78454158e-04, 1.22419500e-03],
       [1.60280470e-04, 9.96303797e-01, 3.53590399e-03],
       [1.10786624e-01, 8.76239121e-01, 1.29742026e-02],
       [9.99571621e-01, 4.01760954e-05, 3.88163724e-04],
       [9.98183787e-01, 1.68534135e-03, 1.30876098e-04],
       [2.92146225e-02, 3.98640707e-03, 9.66798961e-01],
       [1.23847232e-04, 9.60935235e-01, 3.89409959e-02],
       [3.19875926e-02, 4.47733290e-02, 9.23239112e-01],
       [1.55596979e-04, 3.74015130e-04, 9.99470413e-01],
       [2.67243711e-04, 7.78910458e-01, 2.20822290e-01],
       [1.42927596e-03, 3.43231075e-02, 9.64247584e-01],
       [9.95265841e-01, 2.62377